# Spanish to English Translation with SentinelTranslate

This notebook demonstrates how to use SentinelTranslate to translate Spanish text to English using the OPUS-MT (`opus-mt-es-en`) model served via NVIDIA Triton Inference Server.

## What You'll Learn

- Connect to the SentinelTranslate API and verify services are running
- Translate single Spanish sentences to English
- Handle common Spanish phrases (greetings, business, technical)
- Test edge cases (numbers, dates, named entities)
- Understand hallucination detection mechanisms
- Measure translation performance and latency
- Handle errors and implement retry logic

## Prerequisites

**Services must be running**:
```bash
docker-compose up -d
```

**Required models**:
- `opus-mt-es-en` in Triton model repository

**Estimated time**: 15-20 minutes

## Setup and Imports

In [ ]:
import requests
import json
import time
from typing import Dict, Any
from datetime import datetime

# Configuration
SIDECAR_URL = "http://localhost:8080"
BATCH_API_URL = "http://localhost:8090"
TRITON_URL = "http://localhost:8000"

# Language pair
SOURCE_LANG = "es"  # Spanish
TARGET_LANG = "en"  # English
MODEL_NAME = "opus-mt-es-en"

print("✅ Imports successful")
print(f"📝 Source Language: Spanish ({SOURCE_LANG})")
print(f"🎯 Target Language: English ({TARGET_LANG})")
print(f"🤖 Model: {MODEL_NAME}")

## Step 1: Verify Services are Running

Before we start translating, let's make sure all services are healthy.

In [ ]:
def check_service_health(service_name: str, url: str) -> bool:
    """Check if a service is healthy and responding."""
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            print(f"✅ {service_name}: Healthy")
            return True
        else:
            print(f"⚠️  {service_name}: Responded with status {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"❌ {service_name}: Not reachable - {e}")
        return False

# Check all services
print("Checking service health...\n")
sidecar_healthy = check_service_health("Sidecar API", f"{SIDECAR_URL}/health")
batch_healthy = check_service_health("Batch API", f"{BATCH_API_URL}/health")
triton_healthy = check_service_health("Triton Server", f"{TRITON_URL}/v2/health/ready")

if sidecar_healthy and triton_healthy:
    print("\n🎉 All required services are running!")
else:
    print("\n⚠️  Some services are not running. Please start them with: docker-compose up -d")

## Step 2: Helper Functions

Let's create utility functions to simplify translation requests.

In [ ]:
def translate_text(text: str, source_lang: str = SOURCE_LANG, target_lang: str = TARGET_LANG, 
                   max_wait: int = 30, poll_interval: float = 0.5) -> Dict[str, Any]:
    """Translate text using SentinelTranslate API."""
    start_time = time.time()
    
    payload = {
        "text": text,
        "source_lang": source_lang,
        "target_lang": target_lang
    }
    
    try:
        response = requests.post(f"{SIDECAR_URL}/translate", json=payload, timeout=10)
        response.raise_for_status()
        job_data = response.json()
        job_id = job_data["job_id"]
    except Exception as e:
        return {"success": False, "error": f"Failed to submit job: {e}", "source": text}
    
    elapsed = 0
    while elapsed < max_wait:
        try:
            status_response = requests.get(f"{SIDECAR_URL}/status/{job_id}", timeout=5)
            status_response.raise_for_status()
            result = status_response.json()
            
            if result["status"] == "SUCCESS":
                translation_time = time.time() - start_time
                return {
                    "success": True,
                    "source": text,
                    "translation": result["result"],
                    "job_id": job_id,
                    "latency_ms": round(translation_time * 1000, 2),
                    "source_lang": source_lang,
                    "target_lang": target_lang
                }
            elif result["status"] == "FAILURE":
                return {"success": False, "error": result.get("error", "Unknown error"), "source": text, "job_id": job_id}
            
            time.sleep(poll_interval)
            elapsed = time.time() - start_time
            
        except Exception as e:
            return {"success": False, "error": f"Error checking status: {e}", "source": text, "job_id": job_id}
    
    return {"success": False, "error": f"Timeout after {max_wait}s", "source": text, "job_id": job_id}

def print_translation(result: Dict[str, Any]):
    """Pretty print translation result."""
    if result["success"]:
        print(f"🇪🇸 Source:      {result['source']}")
        print(f"🇬🇧 Translation: {result['translation']}")
        print(f"⏱️  Latency:     {result['latency_ms']} ms")
        print(f"🆔 Job ID:      {result['job_id'][:8]}...")
    else:
        print(f"❌ Error: {result['error']}")
        print(f"🇪🇸 Source: {result['source']}")

print("✅ Helper functions defined")

## Step 3: Basic Translation Examples

Let's start with simple Spanish sentences.

In [ ]:
# Simple greeting
print("Example 1: Simple Greeting\n" + "="*50)
result = translate_text("Hola, ¿cómo estás?")
print_translation(result)

In [ ]:
# Longer sentence
print("\nExample 2: Longer Sentence\n" + "="*50)
result = translate_text("Estoy muy feliz de conocerte hoy.")
print_translation(result)

## Step 4: Common Spanish Phrases

Let's translate common phrases you might encounter in different contexts.

In [ ]:
phrases = [
    # Greetings
    "Buenos días",
    "Buenas tardes",
    "Buenas noches",
    "Encantado de conocerte",
    
    # Business
    "Me gustaría programar una reunión.",
    "¿Podría enviarme el informe?",
    "Gracias por su colaboración.",
    "La fecha límite es el viernes.",
    
    # Technical
    "El servidor no responde.",
    "Por favor, reinicie la aplicación.",
    "La base de datos está sobrecargada.",
    "Necesitamos optimizar el rendimiento."
]

print("Common Spanish Phrases\n" + "="*70 + "\n")

latencies = []
for phrase in phrases:
    result = translate_text(phrase)
    if result["success"]:
        print(f"🇪🇸 {phrase}")
        print(f"🇬🇧 {result['translation']}")
        print(f"   ({result['latency_ms']} ms)\n")
        latencies.append(result['latency_ms'])
    else:
        print(f"❌ Failed: {phrase} - {result['error']}\n")

if latencies:
    avg_latency = sum(latencies) / len(latencies)
    print(f"\n📊 Average Latency: {avg_latency:.2f} ms")
    print(f"📊 Min Latency: {min(latencies):.2f} ms")
    print(f"📊 Max Latency: {max(latencies):.2f} ms")

## Step 5: Edge Cases - Numbers and Dates

Spanish uses different number formatting (e.g., 1.299,99 instead of 1,299.99). Let's test number consistency checks.

In [ ]:
number_tests = [
    "Tengo 25 años.",
    "La reunión es a las 14:30.",
    "El precio es de 1.299,99 euros.",
    "Vendimos 10.000 unidades en 2023.",
    "El informe tiene 3 tablas y 15 gráficos.",
    "Mi número de teléfono es 91 234 56 78."
]

print("Edge Cases: Numbers and Dates\n" + "="*70 + "\n")

for test in number_tests:
    result = translate_text(test)
    if result["success"]:
        print(f"🇪🇸 {test}")
        print(f"🇬🇧 {result['translation']}")
        print(f"   ✅ Number consistency check passed\n")
    else:
        print(f"🇪🇸 {test}")
        print(f"⚠️  {result['error']}\n")

## Step 6: Edge Cases - Named Entities

Spanish names and places should be preserved correctly during translation.

In [ ]:
ner_tests = [
    "María trabaja en Google en Madrid.",
    "Pablo Picasso nació en Málaga, España.",
    "La empresa Amazon fue fundada por Jeff Bezos.",
    "Pedro Sánchez es el presidente del gobierno.",
    "La Sagrada Familia está en Barcelona.",
    "Gabriel García Márquez escribió Cien años de soledad.",
    "El Real Madrid y el Barcelona son rivales históricos."
]

print("Edge Cases: Named Entities (NER)\n" + "="*70 + "\n")

for test in ner_tests:
    result = translate_text(test)
    if result["success"]:
        print(f"🇪🇸 {test}")
        print(f"🇬🇧 {result['translation']}")
        print("   ✅ NER consistency check passed\n")
    else:
        print(f"🇪🇸 {test}")
        print(f"⚠️  {result['error']}\n")

## Step 7: Edge Cases - Special Characters and Accents

Spanish uses tildes (ñ), accents (á, é, í, ó, ú), and inverted punctuation (¿¡).

In [ ]:
accent_tests = [
    "Mañana es un día especial.",
    "¿Dónde está la biblioteca?",
    "¡Qué increíble!",
    "El niño estudió para el examen.",
    "Había una vez un español en México.",
    "José y María viven en Córdoba."
]

print("Edge Cases: Special Characters and Accents\n" + "="*70 + "\n")

for test in accent_tests:
    result = translate_text(test)
    if result["success"]:
        print(f"🇪🇸 {test}")
        print(f"🇬🇧 {result['translation']}\n")
    else:
        print(f"🇪🇸 {test}")
        print(f"⚠️  {result['error']}\n")

## Step 8: Hallucination Detection Examples

Test edge cases that might trigger SentinelTranslate's safety checks.

In [ ]:
print("Hallucination Test 1: Very Short Text\n" + "="*50)
result = translate_text("Sí.")
print_translation(result)

print("\n" + "="*70 + "\n")

print("Hallucination Test 2: Long Complex Sentence\n" + "="*50)
long_text = (
    "La transformación digital de las empresas requiere un enfoque estratégico "
    "que combine la innovación tecnológica, la formación de los empleados, y "
    "la adaptación de los procesos de negocio para mantenerse competitivo en "
    "un mercado en constante evolución."
)
result = translate_text(long_text)
print_translation(result)

## Step 9: Performance Benchmarking

In [ ]:
import statistics

benchmark_phrases = [
    "Hola",
    "¿Cómo estás?",
    "Soy desarrollador.",
    "¿Qué tiempo hace?",
    "Muchas gracias."
]

print("Performance Benchmark\n" + "="*70 + "\n")
print("Running 20 translations (4 iterations of 5 phrases)...\n")

all_latencies = []
start_time = time.time()

for iteration in range(4):
    for phrase in benchmark_phrases:
        result = translate_text(phrase)
        if result["success"]:
            all_latencies.append(result['latency_ms'])

total_time = time.time() - start_time

if all_latencies:
    print(f"✅ Completed {len(all_latencies)} translations in {total_time:.2f} seconds\n")
    print(f"📊 Average Latency: {statistics.mean(all_latencies):.2f} ms")
    print(f"📊 Median Latency:  {statistics.median(all_latencies):.2f} ms")
    print(f"📊 Min Latency:     {min(all_latencies):.2f} ms")
    print(f"📊 Max Latency:     {max(all_latencies):.2f} ms")
    print(f"📊 Std Deviation:   {statistics.stdev(all_latencies):.2f} ms")
    print(f"\n📊 Throughput:      {len(all_latencies) / total_time:.2f} translations/second")

## Summary

In this notebook, you learned how to translate Spanish to English using SentinelTranslate, including:

✅ Basic translation workflow  
✅ Common Spanish phrases and business terminology  
✅ Handling Spanish-specific characters (ñ, accents, inverted punctuation)  
✅ Edge cases with numbers and named entities  
✅ Hallucination detection mechanisms  
✅ Performance measurement  

## Next Steps

- Explore other Romance languages (French, Portuguese, Italian)
- Try different script families (Cyrillic, Arabic, Chinese)
- Use batch translation for large datasets
- Convert additional OPUS-MT models for your needs

---

**Model**: `opus-mt-es-en` from [Helsinki-NLP/OPUS-MT](https://github.com/Helsinki-NLP/Opus-MT)  
**License**: CC-BY-4.0